In [2]:
import requests
import json
import pandas as pd

In [13]:
def datagovsg_api_call_v2(resource_id: str='d_8b84c4ee58e3cfc0ece0d773c8ca6abc'
                            , verbose: bool = True, **kwargs) -> pd.DataFrame:
    '''
    Function to build the API call and construct the pandas dataframe, 
    data.gov now limits the calls, so it is safer to call month by month to not have data truncated
    ## Parameters
    resource_id: str
        resource_id for API
    verbose: bool
            whether to print out the calls
    **kwargs:
        refer to API guide https://guide.data.gov.sg/developers/api-v2
        e.g.:
            year: str
                year
            month: 
                month desired, int between 1-12
            sort: str
                field, by ascending/desc, default by Latest month
            limit: int
                maximum entries (API default by datagov is 100, if not specified)
            
    Returns Dataframe of data : pd.DataFrame
    '''
    payload = {}
    if kwargs.get("year") and kwargs.get("month"):
        year_month = f'{kwargs.get("year")}-{str(kwargs.get("month")).zfill(2)}'
        month_filter = {"month": [year_month]}
        # To add a nested dictionary for request get, we need to parse the nested dictionary into json format
        payload["filters"] = json.dumps(month_filter)
    payload["limit"] = 10000
    payload["sort"] = 'month desc'

    url = f"https://data.gov.sg/api/action/datastore_search?resource_id={resource_id}"
    response = requests.get(url, params=payload)
    response.raise_for_status()
    if verbose:
        print(f'Get request call: {response.url}')
    data = response.json()
    df = pd.DataFrame(data['result']['records'])
    return df

In [12]:
datagovsg_api_call_v2(year=2024, month=1)

Get request call: https://data.gov.sg/api/action/datastore_search?resource_id=d_8b84c4ee58e3cfc0ece0d773c8ca6abc&filters=%7B%22month%22%3A+%5B%222024-01%22%5D%7D&limit=10000&sort=month+desc


,_id,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price
0,169172,2024-01,ANG MO KIO,2 ROOM,116,ANG MO KIO AVE 4,07 TO 09,44,Improved,1978,53 years 06 months,288000
1,169173,2024-01,ANG MO KIO,2 ROOM,510,ANG MO KIO AVE 8,04 TO 06,44,Improved,1980,55 years 07 months,322500
2,169174,2024-01,ANG MO KIO,3 ROOM,308B,ANG MO KIO AVE 1,01 TO 03,70,Model A,2012,87 years 09 months,520000
3,169175,2024-01,ANG MO KIO,3 ROOM,308B,ANG MO KIO AVE 1,25 TO 27,70,Model A,2012,87 years 09 months,650000
4,169176,2024-01,ANG MO KIO,3 ROOM,223,ANG MO KIO AVE 1,04 TO 06,67,New Generation,1978,53 years 01 month,343800
...,...,...,...,...,...,...,...,...,...,...,...,...
1269,170441,2024-01,YISHUN,5 ROOM,602,YISHUN ST 61,07 TO 09,121,Improved,1987,62 years 05 months,688000
1270,170442,2024-01,YISHUN,5 ROOM,820,YISHUN ST 81,04 TO 06,122,Improved,1988,63 years 08 months,670002
1271,170443,2024-01,YISHUN,5 ROOM,866,YISHUN ST 81,04 TO 06,122,Improved,1988,63 years 02 months,675000
1272,170444,2024-01,YISHUN,EXECUTIVE,356,YISHUN RING RD,04 TO 06,146,Maisonette,1988,63 years 08 months,860000
